In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   null|                B00014|
|     B00021         |2019-10-01 00:00:4

In [8]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql.functions import col

# Assuming you have a SparkSession named spark
# spark = SparkSession.builder.appName("example").getOrCreate()

# Assuming you have the DataFrames fhv_data and fhv_zone
# Replace the placeholder paths with your actual file paths or DataFrame references
fhv_data_schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),  # Assuming PULocationID is available in fhv_data
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

fhv_zone_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

fhv_data_df = spark.read.csv('/path/to/fhv_data.csv', header=True, schema=fhv_data_schema)
fhv_zone_df = spark.read.csv('/path/to/fhv_zone_data.csv', header=True, schema=fhv_zone_schema)

# Replace 'zone' with the actual column name from your SQL query
df_result = spark.sql("""
SELECT 
    PULocationID AS pickup_zone,
    DOLocationID AS dropoff_zone
FROM
    fhv_data
GROUP BY
    PULocationID, DOLocationID
""")

# Joining the result of the SQL query with fhv_data and fhv_zone
joined_df = df_result.join(fhv_data_df, df_result.pickup_zone == fhv_data_df.PULocationID, 'inner') \
                     .join(fhv_zone_df, df_result.pickup_zone == fhv_zone_df.LocationID, 'inner')

# Count the occurrences of each pickup location zone
zone_counts = joined_df.groupBy('pickup_zone').count()

# Find the least frequent pickup location zone
least_frequent_zone = zone_counts.orderBy('count').first()['pickup_zone']

print(f"The least frequent pickup location zone is: {least_frequent_zone}")


AnalysisException: Column 'PULocationID' does not exist. Did you mean one of the following? []; line 3 pos 5;
'Aggregate ['zone], ['PULocationID AS pickup_zone#193, 'DOLocationID AS dropoff_zone#194, 'FROM AS fhv_data#195]
+- OneRowRelation


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql.functions import col

# Assuming you have the DataFrames fhv_data and fhv_zone
# Replace the placeholder paths with your actual file paths or DataFrame references
fhv_data_schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

fhv_zone_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

fhv_data_df = spark.read.csv('fhv_tripdata_2019-10.csv', header=True, schema=fhv_data_schema)
fhv_zone_df = spark.read.csv('taxi_zone_lookup.csv', header=True, schema=fhv_zone_schema)

# Joining the result of the SQL query with fhv_data and fhv_zone
joined_df = df_result.join(fhv_data_df, df_result.pickup_zone == fhv_data_df.PULocationID, 'inner') \
                     .join(fhv_zone_df, df_result.pickup_zone == fhv_zone_df.LocationID, 'inner')

# Count the occurrences of each pickup location zone
zone_counts = joined_df.groupBy('pickup_zone').count()

# Find the least frequent pickup location zone
least_frequent_zone = zone_counts.orderBy('count').first()['pickup_zone']

print(f"The least frequent pickup location zone is: {least_frequent_zone}")


NameError: name 'df_result' is not defined